# Clase 12 - Análisis Exploratorio de Datos

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

**Profesor: Matías Rojas**

## Objetivos de la Clase 

![Etapas de un Proyecto de Ciencia de Datos](https://i.ibb.co/N2rNjrZ/etapas.png)

Como hemos visto anteriomente, los datos son la materia prima para la contrucción de análisis estadísticos y modelos predictivos. Esto implica que las hipótesis y predicciones  elaboradas por dichos modelos estarán directamente relacionados con la calidad de los datos. Por ende, es crítico asegurar un preprocesado y una buena examinación de los datasets a trabajar.

El contenido de esta cátedra, se centra en las técnicas esenciales para el preprocesado de datos usando las herramientas que hemos visto hasta el momento. 




## Análisis Exploratorio de Datos

El análisis exploratorio de los datos (Exploratory data analysis [EDA] en inglés) consiste en realizar investigaciones iniciales sobre los datos **con el fin de descubrir sus patrones y anomalías, como también testear hipótesis que tengamos sobre estos**. Comunmente este análisis va de la mano de la **estadística descriptiva y de las visualizaciones**, que es lo que hemos estado trabajando hasta el momento en el curso.



    
<img src='https://i.ibb.co/5Fsrbxn/eda-related.png' alt='Gráficos en análisis exploratorio de datos'/>

<div align='center'/>
Análisis exploratorio de datos. 
<br>
Fuente: <a href='https://towardsdatascience.com/exploratory-data-analysis-8fc1cb20fd15'>What is Exploratory Data Analysis?</a>
</center>



En general, podemos resumir este proceso en estas 4 etapas: 

1. **Entender el problema y el dataset con el que trabajaremos:** Este punto se aleja un poco del curso pero de todas maneras hay que tener clara la importancia del entendimieto del negocio/investigación.
2. **Análisis Univariado**: Estudiar los patrones de las variables a través de estadísticas de resumen y análisis de la distribución de los datos. Aquí los datos se estudian de manera individual, sin considerar relaciones con otras variables.
3. **Análisis Multivariado**: Estudiar como se comporta una variable con respecto a las otras.
4. **Análisis y Limpieza de los Datos Faltantes.**




### Repaso: Estadísticas de Resumen


Las estadísticas de resumen son valores que explican propiedades y patrones de los datos. 
Ejemplo: 
    
- Medidas de tendencia central (ver como se comportan los datos a través de valores centrales) como la media, mediana y moda.
- Dispersión (miden la variabilidad de los datos) como el rango (máximo - mínimo), la desviación estándar y la varianza. 
- Percentiles y Cuartiles.

En el caso en que estemos analizando de forma multivariada, podemos usar: 

- Correlación (mide el grado de asociación lineal entre dos variables).
- Tablas de contingencia (conteo entre ocurrencias de variables categóricas).


Información más teórica sobre estos temas la pueden encontrar en la siguiente clase de minería de datos: https://github.com/dccuchile/CC5206/blob/master/slides/Clase_3_explora.pdf


## Problema de Esta Clase: House Pricing



![House Pricing](https://storage.googleapis.com/kaggle-competitions/kaggle/5407/media/housesbanner.png)

El dataset **`house pricing`** consiste en 80 variables (79 variables explicativas más una variable objetivo) que describen aspectos fundamentales de hogares residenciales en la ciudad de *Ames, Iowa*. 

La variable objetivo de este dataset es generar un modelo que prediga el precio final de cada hogar por medio de una regresión.

Es un problema "ultra conocido"y pueden encontrar su descripción aquí: https://www.kaggle.com/c/house-prices-advanced-regression-techniques

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import norm

# El conjunto a trabajar es el de entrenamiento
df = pd.read_csv("https://raw.githubusercontent.com/maranedah/MDS7202/main/clases/Clase%2012%20-%20An%C3%A1lisis%20Exploratorio%20de%20Datos%20(EDA)/recursos/train.csv", index_col="Id")
df

In [ ]:
# Preprocesamiento

# Etapa 1: Convertir mes y año a Datetime
df["DateSold"] = pd.to_datetime(
    "1/" + df["MoSold"].astype(str) + "/" + df["YrSold"].astype(str), yearfirst=False
)

# Etapa 2: Generar cuartiles para los precios
df["SalePriceQCut"] = pd.qcut(
    df["SalePrice"], 4, ["Low", "Low-Mid", "Mid-High", "High"]
)

# Etapa 3: Convertir strings a categorías
df = pd.concat(
    [
        df.select_dtypes([], ["object"]),
        df.select_dtypes(["object"]).apply(pd.Series.astype, dtype="category"),
    ],
    axis=1,
)

In [ ]:
df.columns

In [ ]:
df = df.loc[
    :,
    [
        # Terreno
        "LotArea",      # Area del terreno
        "LandSlope",    # Pendiente del terreno
        "Neighborhood", # Barrio
        
        # Metadatos de la Vivienda
        "BldgType",     # Tipo de vivienda
        "YearBuilt",    # Año de construcción
        "YearRemodAdd", # Año de remodelación
        "Utilities",    # Agua, luz, etc...
        
        # Materiales
        "Foundation",   # Fundación de la vivienda
        "RoofMatl",     # Material del techo
        "RoofStyle",    # Estilo del techo
        "Exterior1st",  # Material del Exterior
        "ExterCond",    # Condición del material exterior
        
        # Interior de la casa
        "GrLivArea",    # Area habitable sobre el nivel del suelo.
        "1stFlrSF",     # Area primer piso
        "2ndFlrSF",     # Area segundo piso
        "FullBath",     # Baños completos
        'HalfBath',     # Baños de visita?
        "BedroomAbvGr", # Piezas
        "KitchenAbvGr", # Cocinas
        "KitchenQual",  # Calidad de la cocina
        
        # Sótano
        "TotalBsmtSF",  # Total sótano
        "BsmtCond",     # Condición del sótano
        
        # Garaje
        "GarageType",   # Tipo de garaje
        "GarageCars",   # Cantidad de autos por garaje
        
        # Piscina
        "PoolArea",     # Area de la piscina
        "PoolQC",       # Calidad de la piscina
        
        # Calefacción y Aire
        "Heating",      # Calefacción
        "HeatingQC",    # Calidad de la Calefacción
        "CentralAir",   # Aire Acondicionado Central
        
        # Calidad y Condición
        "OverallQual",  # Calidad general
        "OverallCond",  # Condición general actual
        
        # Datos de la venta
        "DateSold",     # Mes y Año de venta
        "SaleType",     # Tipo de venta
        "SaleCondition",# Condición de la vivienda en la venta
        "SalePrice",    # Precio de la venta
        "SalePriceQCut",# Cuartiles del precio de la vivienda
    ],
]

numeric = df.select_dtypes(["int64"]).columns
categorical = df.select_dtypes(["category"]).columns

In [ ]:
df.info()

In [ ]:
df.head()

---

## 1. Perfilamiento Univariado

El punto inicial para comprender la naturaleza de una variable, pasa por sumarizar sus valores usando estadísticas de resumen, caracterizar su distribución y (en el caso que aplique) analizar su relación con respecto a la variable objetivo. 

El objetivo final de esta etapa es comprender qué papel juega cada variable (de forma independiente) en la resolución del problema.

### Análisis de la Variable Dependiente 

El paso inicial es saber como se distribuye la variable dependiente, es decir, el precio de las casas.


In [ ]:
df[["SalePrice"]].describe()

Este primer análisis lo haremos a través de un histograma en conjunto con un boxplot.


1. Histograma: Permite ver la distribución de los datos.
2. Boxplots: Se construyen usando los cuartiles. 
  - La caja muestra el primer y tercer cuartil. 
  - La mediana está mostrada en la división de la caja.
  - El ancho de la caja es el rango intercuantílico (Q3 - Q1). 
  - Los brazos representan el rango intercuantílico * +/- 1.5. Los valores fuera de este rango pueden ser considerados datos fuera de la norma / outliers.

In [ ]:
import plotly.express as px

px.histogram(
    df, 
    x="SalePrice", 
    title="Análisis de la variable SalePrice", 
    marginal="box"
)

Bajo estos gráficos podemos concluir que:
    - La mayoría de las propiedades se concentran cerca del promedio de la distribución.
    - Existen algunas propiedades muy caras que hacen que la cola de la distribución sea muy larga hacia la derecha. Esto se confirma con el boxplot, en donde existen muchos puntos outliers.
    - Es relativamente uniforme a lo largo del dominio, salvo por algunas discontinudades, como por ejemplo en 170k a 180k.
    - 
    

### Caracterizar distribuciones

Existen algunas medidas útiles para caracterizar la distribución de una variable (aplican bien cuando las distribuciones son normales).

1. **Skewness**: Permite establecer el grado de simetría (o asimetría) que presenta una distribución de probabilidad de una variable aleatoria sin tener que hacer su representación gráfica. Coloquialmente, sería ver hacia que lado está inclinado el gráfico. En el caso anterior sería hacia el lado izquierdo.

<div align='center'>
    <img src='https://i.ibb.co/QYf6ZPF/skew.pngg' alt='Skewness' width=600/>
</div>

2. **Curtosis**: Determina el grado de concentración que presentan los valores en la región central de la distribución.

<div align='center'>
    <img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT1n1f_pfBRWXsET7Zk8UbRnzKLDPqsBs5ESL_uh9aKmlDcIpkO' alt='Kurtosis' width=600/>
</div>


In [ ]:
# skewness and kurtosis
print("Skewness: %f" % df["SalePrice"].skew())
print("Kurtosis: %f" % df["SalePrice"].kurt())

### Distribuciones de Variables Numéricas

A continuación, generaremos histogramas que nos permitirán entender y visualizar la geometría de cada distribución.

In [ ]:
numeric

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

n_cols = 4

fig = make_subplots(
    rows=len(numeric) // n_cols + 1, cols=n_cols, subplot_titles=numeric
)

for idx, col in enumerate(numeric):
    row_idx = idx // n_cols + 1
    col_idx = idx % n_cols + 1
    
    hist = go.Histogram(x=df.loc[:, col], name=col, histnorm="probability")
    fig.add_trace(hist, row=row_idx, col=col_idx)

fig.update_layout(
    height=800,
    title_text="Análisis Univariado de las Variables Numéricas",
    showlegend=False,
)
fig.show()

### Distribuciones de Variables Categóricas

Para las variables categóricas, se genera un conteo de valores únicos.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

n_cols = 4

fig = make_subplots(
    rows=len(categorical) // n_cols + 1, cols=n_cols, subplot_titles=categorical
)

for idx, col in enumerate(categorical):
    row_idx = idx // n_cols + 1
    col_idx = idx % n_cols + 1

    data = df.loc[:, col]
    hist = go.Histogram(x=data, name=col)
    fig.add_trace(hist, row=row_idx, col=col_idx)

fig.update_layout(
    height=1200,
    title_text="Análisis Univariado de las Variables Categoricas",
    showlegend=False,
)
fig.show()

Al observar las distribuciones, es importante buscar si existe variabilidad dentro de estas, pues por lo general, una variable con un único valor casi seguro, no aporta información a la dinámica de los datos.



## 2. Perfilamiento Multivariado

La idea del perfilamiento multivariado es estudiar las relaciones entre las distintas variables del dataset.


### Scatter Matrix

Una scatter matrix nos permite ver relaciones entre las distintas variables que componen un dataset a través de múltiples scatter plots:

In [ ]:
variables_interes = [
    "GrLivArea",
    "TotalBsmtSF",
    "1stFlrSF",
    "2ndFlrSF",
    "FullBath",
    "YearBuilt",
    "OverallQual",
]

In [ ]:
df.columns

In [ ]:
fig = px.scatter_matrix(
    df.sort_values(by="SalePriceQCut", ascending=True),
    dimensions=variables_interes,
    height=1200,
    title="Análisis Bivariado",
    color='SalePriceQCut'
)
fig.update_traces(diagonal_visible=False)
fig.show()

La última fila de esta visualización entrega una idea de la relación entre las variables de interés.  Dentro de las interacciones entre variables, se observa que 'GrLivArea' y 'TotalBsmtSf' se comportan de manera similar contra 'OverllQuall', esperandose cierta tendencia creciente en ambos casos.

### Análisis de las Variables Independientes con Respecto a la Variable Dependiente


#### Gráficos de violín

Un gráfico de violín permite sumarizar y observar características de un dataset. Este se comporta como un gráfico de cajas (boxplot), mostrando la mediana, el rango intercuantílico IQR (percentil 75 - percentil 25, o Q3 - Q1) y el rango 1.5 intercuantílico (Q3 +- 1.5 IQR). Además de lo anterior, se suma una estimación de la densidad por kernel a cada lado. Esto quiere decir, que zonas con mayor densidad, se verán como 'montes' horizontales. 

#### OverallQual

Se observa la variable 'OverallQual' (categórica) y se compara con la variable de interés 'SalePrice'. Para ello se usa un gráfico de categórias tipo violín y box (para adicionalmente comparar ambas visualizaciones)

In [ ]:
import plotly.express as px

fig = make_subplots(rows=2, cols=1)

fig_1 = px.violin(
    df, x="OverallQual", y="SalePrice", title="Violin plot OverallQuall vs SalePrice"
)
fig.add_trace(fig_1.data[0], row=1, col=1)

fig_2 = px.box(df, x="OverallQual", y="SalePrice")

fig.add_trace(fig_2.data[0], row=2, col=1)

fig.update_layout(height=800)
fig.show()

#### LandSlope

Por otra parte, analizando las gráficas univariadas, se puede observar que para 'LandSlope', se tiene poca variablidad y no genera diferencias en distribución para 'SalePrice' en ninguna de sus categorias. 

In [ ]:
fig = px.violin(
    df, x="LandSlope", y ="SalePrice", title="Violin plot LandSlope vs SalePrice"
)
fig.show()

> **Ejercicio ✏️**

1. Los gráficos generados anteriormente siguen exactamente el mismo patrón de generación, lo único que cambia es la columna a analizar. Esto es una mala práctica pues siempre se debe buscar reutilizar código o 'no repetirse' esto se conoce como principio **DRY (don't repeat yourself)**. Construya una función que permita visualizar columnas categóricas del dataset y compararlas con 'SalePrice'. 

2. En función de las visualizaciones construidas, discuta que variables categóricas pueden ser de interés para predecir 'SalePrice'. Busque variabilidad y separación en la distribución de precios. ¿Qué ocurre si una variable categórica posee poca variablidad pero genera buenas separaciones en  'SalePrice'?

#### Gráficos de Dispersión o Scatterplots

Para comparar las variables numéricas, se pueden utilizar gráficos de dispersión contra 'SalePrice'. En este caso, se buscan variabilidad en el histograma univariado y a la vez, se buscan relaciones funcionales (del tipo lineal, exponencial, cuadrático, etc..) con 'SalePrice'. 


Por ejemplo, estudiemos la variable 'GrLivArea'. Para esto, primero definimos una función para gráficar variables numéricas con respecto a 'SalePrice'.

In [ ]:
def scatter_dists(numeric_col, trend=None):
    """Dibuja un scatterplot de una columna con respecto a SalePrice.

    numeric_col:str
    Nombre de la columna

    trendline: str
    (Original de plotly) One of `'ols'` or `'lowess'`. If `'ols'`, an Ordinary Least Squares
    regression line will be drawn for each discrete-color/symbol group. If
    `'lowess`', a Locally Weighted Scatterplot Smoothing line will be drawn
    for each discrete-color/symbol group.
    """
    fig = px.scatter(
        df,
        x=numeric_col,
        y="SalePrice",
        title=f"{numeric_col} vs SalePrice",
        trendline=trend,
    )
    fig.show()

In [ ]:
scatter_dists("LotArea", trend="ols")

En este caso, se puede observar una distrbución univariada bien definida y un comportamiento lineal aunque ruidoso. Esto hace que 'GrLivArea' sea una variable de interés. De la misma manera, '1stFlrSF', parece reflejar las mismas buenas características. 



In [ ]:
scatter_dists("1stFlrSF", trend="ols")

En el caso de 'TotalBsmtSF' se tiene

In [ ]:
scatter_dists("TotalBsmtSF", trend="ols")

Una relación menos lineal con un poco más de ruido pero una buena distribución en e dataset. Esta variable puede ser de interés pero esto se puede estudiar a posteriori. 

Finalmente para 'MasVnrArea', se tiene

In [ ]:
scatter_dists("PoolArea", trend="ols")

Se aprecia una distribución altamente concentrada en el 0 y poco relacionada con la variable a predecir.

> **Ejercicio ✏️**:

1. Estudie las siguientes proposiciones: 

    1.'OverallQual' y 'YearBuilt' parecen relacionadas con 'SalePrice'. 
    2. En el caso de 'OverallQual', esta relación es bastante débil.
    3. En el caso de 'YearBuilt', esta relación es bastante débil.
    4. Los gráficos de caja para 'OverallQual contra  'SalePrice' muestran cierta linealidad con respecto a 'SalePrice'.

2. Estudie la distribución univariada de 'SalePrice', a continuación ejecute el test K^2 de D’Agostino usando `normaltest` del módulo `stats` de SciPy. Compare para una significancia de 5%. ¿ Se puede tratar esta variable como distribuida de manera normal, tomando en cuenta su comportamiento estadístico?

3. Las distribuciones de 'TotalBsmtSF' y '1stFlrSF' parecen bastante similares, más aún sus relaciones con 'SalePrice' comparten una tendencia. Ejecute el [test de Kolmogorov-Smirnov ](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ks_2samp.html) por medio de `ks_2samp` para explorar la hipótesis:

 ```'TotalBsmtSF' y '1stFlrSF' vienen de la misma distribución```
 
4. Estudie algunos estadísticos de interés según el tipo de dato.
    
    1. Para las variables numéricas estudie promedios, desviaciones estándar y rangos intercuartílicos. Utilice los rangos calculados para tener una idea del porcentaje de valores fuera de tales rangos por columna. 
    
    2. Para variables catégoricas calcule frecuencias, proporciones y modas. Utilice lo anterior para obtener alguna idea de la variabilidad de los datos.
    

#### Categorías Paralelas

Permite comparar el comportamiento de variables categoricas a través de un gráfico 2d. Cada categoría es representada por una barra y conjuntos de datos similares se agrupan en cintas (*ribbons*).

In [ ]:
df.info()

In [ ]:
fig = px.parallel_categories(
    df.sample(1000).sort_values("SalePriceQCut", ascending=False),
    dimensions=[
        "OverallQual",
        "Exterior1st",
        "Utilities",
        "SalePriceQCut",
    ],
)
fig.show()

> **Ejercicio ✏️**: Explore el equivalente de Categorías Paralelas para variables numéricas ([Coordenadas paralelas](https://plotly.com/python/parallel-coordinates-plot/)) y grafique la relación entre variables numéricas y 'SalePrice'.

### Análisis Temporal

Este análisis nos permite ver como se comportan los valores de alguna variable a través del tiempo.

In [ ]:
px.box(
    df.query("YearBuilt > 1920").sort_values("YearBuilt"), y="SalePrice", x="YearBuilt"
)

### Correlaciones entre variables

Los análsis iniciales basados en visualizaciones sirven para comprender a grandes rasgos la estructura del dataset. Este tipo de exploración debe ser acompañada de tests estadísticos como los vistos en los ejercicios anteriores. En el caso del perfilamiento bivariado se puede usar una técnica mixta, basada en el análisis de las correlaciones.

>  La correlación indica la fuerza y la dirección de una relación lineal entre dos variables estadísticas. Ojo:  La correlación entre dos variables no implica, por sí misma, ninguna relación de causalidad.

Podemos calcular la matriz de correlaciones por variables numericas usando el método `corr()`.

In [ ]:
corrmat = df.corr()

In [ ]:
corrmat

se muestran las dos variables más correlacionadas (positivamente) con 'SalePrice'.

Y lo podemos graficas usando `px.imshow`

In [ ]:
# Univariado: Cada variable con respecto al precio

px.imshow(
    corrmat.loc[["SalePrice"], :].sort_values("SalePrice", axis=1),
    zmin=-1,
    zmax=1,
    color_continuous_midpoint=0,
    color_continuous_scale=px.colors.sequential.Viridis,
)

Si bien, las correlaciones con 'SalePrice' deben ser analizadas con más detenimiento, se pude ver claramente que 'GrLivArea', 'TotalBsmtSF', y 'OverallQual' juegan un papel preponderante en el valor de la propiedad.

In [ ]:
px.imshow(
    corrmat,
    aspect="16:9",
    title="Correlación entre Variables",
    height=800,
    zmin=-1,
    color_continuous_midpoint=0,
    zmax=1,
    color_continuous_scale=px.colors.sequential.Viridis,
)

Según el esquema de valores, se buscan los puntos más claros y más oscuros fuera de la diagonal. En primera instancia, las variables  'TotalBsmtSF' y '1stFlrSF' parece bastante correlacionadas, lo mismo ocurre con la variables 'GarageCars' y 'GarageArea', esto puede indicar multicolinearidad que implica información duplicada o relacionada de manera trivial en el dataset. 

> **Ejercicio ✏️**: Obtenga las 15 correlaciones más altas (positiva o negativa) con 'SalePrice'.


### Tablas de Contingencia

Para analizar valores categóricos (categórico vs categórico) existen herramientas especializadas una de ellas es por medio de tablas de dos tratamientos o de contingencia (2 way tables). 

Estas permiten calcular el numero de ocurrencias de una variable para cada una de sus categorías en comparación con los valores de otra variable.

Por ejemplo, construyamos una tabla para analizar 'OverallQual' vs 'GarageCars':

In [ ]:
# Se construye la tabla
tabla = pd.crosstab(
    index=df["OverallQual"],
    columns=df["GarageCars"],
)
tabla

Para el caso de  'OverallQual' y 'GarageCars' vemos que tienden a acumularse dentro de una rango reducido, se puede concluir que a medida que 'OverallQual' crece entre 4 y 6, aparece un aumento considerable en la categoría 'GarageCars' hasta que esta última llega al valor 2, valores superiores paracieran ser independientes de 'OverallQual'. 

> **Ejercicios ✏️**

1. Compare variables categóricas usando este método, ¿se puede encontrar alguna relación entre categórias?

2. Es posible aplicar este método para comparar variables categóricas y continuas, para esto se necesita categorizar la variable continua objetivo. utilice 'SalePriceQuantile' en 5 tramos para compararla con 'OverallQual' ¿Se observa alguna tendencia?


## 3. Manejo de datos faltantes 

Los métodos estándar de manejo de datos han sido desarrollados para para analizar arreglos tabulares. Por lo general las filas de tal arreglo representan observaciones y las columnas sus características asociadas. Cada entrada en este arreglo puede ser modelada como un número, siendo este ligado a un proceso subyacente continuo o discreto. Para comprender tal proceso, es de utilidad sumarizar y observar los valores faltantes con el fin de obtener patrones y seleccionar estrategias para tratarlos. 

### Exploración de valores faltantes

Cuando los datos faltantes se encuentran en variables que no son de interés, se pueden obviar. Sin embargo, esto no es la regla general.


In [ ]:
df.isnull()

In [ ]:
df.isnull().sum()

Sin embargo, basta observar las columnas para comprender que tales variables si poseen valores faltantes

In [ ]:
var_missing = ["BsmtCond", "GarageType", "PoolQC"]

df[var_missing].head()

Por tal motivo es necesario realizar una exploración inicial de los datos faltantes en conjunción con los análisis de distribución iniciales. 

**Ejercicios ✏️**

1. Estudie la distribución de los valores faltantes en las variables numéricas.

2. Considerando que para las variables categóricas las variables con valor 'nan' son consideradas como una nueva categoría. ¿Se ven afectados los análisis anteriores sobre sus distribuciones?

**Ejemplo**

Para estudiar en mayor profundidad la distribución de los valores faltantes, se procede a transformarlos en formato `np.nan`

In [ ]:
df.replace("nan", np.nan, inplace=True)

In [ ]:
px.bar(df.isnull().sum().sort_values())

### Nullity Matrix 

En términos generales, los valores perdidos de este dataset se encuentran relativamente limpios pues están estandarizados con la categoría 'nan'.


Dado que sumarizar valores faltantes genera una estructura de datos, vale la pena explorarla visualmente, para facilitar tal tarea, existe la librería `missingno`

In [ ]:
# Instalamos el paquete usando conda o pip
import sys

!pip install missingno
# !{sys.executable} -m pip install numpy # descomentar si se usa pip
import missingno as msno

Las visualizaciones de generadas por medio de esta librería pueden ser utilizadas para discutir el problema de valores faltantes y generan una estrategia para su tratamiento.

In [ ]:
df.isnull().sum().nlargest(10)

mediante la libreria `missingno` es posible ver el panorama completo de los valores faltanes en el dataset de manera sencilla 

In [ ]:
import matplotlib.pyplot as plt
import missingno as msno

fig, ax = plt.subplots(figsize=[15, 10])

msno.matrix(df, ax=ax, sparkline=False)

**Ejercicio ✏️**

1. Genere un subconjunto con las 10 columnas con mayor información faltante y genere el gráfico anterior sin usar un objeto `axes` y con la opción `sparkline=True`.

Esta visualización muestra que exiten columnas practicamente sin información, según la agregación anterior, estas corresponden a 'PoolQC', y 'BsmtCond'

Por medio de correlaciones entre valores faltantes, es posible obtener un análisis bivariado análogo al anteriormente generado. Para ello se puede utilizar un mapa de calor.

In [ ]:
fig, ax = plt.subplots(figsize=[15, 10])
msno.heatmap(df, ax=ax)

plt.show()

Este gráfico muestra correlaciones de nulidad entre pares de variables, estas varian desde -1 a 1, donde

- $-1$ significa que las variables son excluyentes, es decir, la aparición de una hace que la otra este ausente. 
- $1$ corresponde inclusión, esto quiere decir, que la aparición de una hace que la otra aparezca. Valores cercanos a 0 (sin valor numérico en el gráfico) indican ausencia de relación de nulidad entre las variables.

En el gráfico recien generado, no se observen relaciones de nulidad negativa, por otra parte, existen variables fuertemente relacionadas en cuanto a su información como lo son 'MasVnrType' y 'MasVnrArea', el comportamiento general es que la información esta fuertemente relacionada (en el sentido de inclusión de información) o simplemente no lo está. 


> **Ejercicio ✏️**

1. El gráfico de correlaciones de nulidad permite tener una idea de como se relaciona la información faltante en pares de variables. Para comparar más de dos variables es posible utilizar un *dendograma*. Utilice las 20 variables con mayor cantidad de valores faltanes visualice su dendograma por medio de `msno.dendogram`. Interprete los resultados.[*Hint*](https://github.com/ResidentMario/missingno)




> **Pregunta ❓**: ¿Qué hacemos con las columnas con datos faltantes?

---

## 4. `Pandas Profiling`


Pandas Profiling es una herramienta de perfilamiento automático de variables, el cuál dado un dataframe, genera un reporte en formato HTML.
El reporte contiene información detallada sobre cada variable más gráficos de interacciones y correlaciones entre estas. Según la documentación, los perfiles generados contienen: 


- **Type inference**: detect the types of columns in a dataframe.
- **Essentials**: type, unique values, missing values
- **Quantile statistics** like minimum value, Q1, median, Q3, maximum, range, interquartile range
- **Descriptive statistics** like mean, mode, standard deviation, sum, median absolute deviation, coefficient of variation, kurtosis, skewness
- **Most frequent values**
- **Histogram**
- **Correlations** highlighting of highly correlated variables, Spearman, Pearson and Kendall matrices
- **Missing values matrix**, count, heatmap and dendrogram of missing values
- **Text analysis** learn about categories (Uppercase, Space), scripts (Latin, Cyrillic) and blocks (ASCII) of text data.
- **File and Image analysis** extract file sizes, creation dates and dimensions and scan for truncated images or those containing EXIF information.


Bien usada, es una herramienta muy poderosa que les permitirá acelerar mucho el Análisis Exploratorio de Datos.
Sin embargo, simpre estará la posibilidad de que tengan que programar algo "a mano" ya que el análisis entregado por el profiler no es suficiente.

In [ ]:
!pip install ipywidgets
!pip install --upgrade pandas_profiling

In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(
    df,
    title="Reporte Housing Pricing",
    explorative=True,
    vars={"num": {"low_categorical_threshold": 0}} # resolver problema #954
)

# Problema 954:
# https://github.com/ydataai/pandas-profiling/issues/954

profile.to_file("report.html")

In [ ]:
profile